# Evaluate effectiveness of FilterFinder  
Thomas Macrina  
May 17, 2017  

Inspecting meshsets for FilterFinder evaluation.  

Evaluating on gcloud with instance `alembic-davittest-across`  

First batch of meshsets was copied from here:  
`gs://image_assembly/experiments/davit_piritest/meshset_inspected/standard_224`  

In [1]:
using Alembic

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in anonymous at no file
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:320
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:320
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:320
 in require at ./loading.jl:259
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:320
 in require at ./loading.jl:259
 in include_string at loading.jl:282
 in execute_request at /home/ubuntu/.julia/v0.4/IJulia/src/execute_request.jl:164
 in eventloop at /home/ubuntu/.julia/v0.4/IJulia/src/IJulia.jl:138
 in anonymous at task.jl:447
while loading /home/ubuntu/.julia/v0.4/MKLSparse/src/./DSS/dss_generator.jl, in expression starting on line 3
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in anonymous at no file
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:320
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:320
 in inc

1,1002-1,1096_aligned.jls  
1,2002-1,2096_aligned.jls  
1,2-1,96_aligned.jls  
1,3002-1,3096_aligned.jls

0-thousands: original  
1-thousands: net trained on adj  
2-thousands: net trained on across  
3-thousands: bandpass

In [2]:
ms_dir = joinpath(homedir(), "davit", "standard_224_annotated")
ms_filenames = Dict(
    "original" => "1,2-1,96_aligned.jls",
    "net_adj" => "1,1002-1,1096_aligned.jls",
    "net_across" => "1,2002-1,2096_aligned.jls",
    "bandpass" => "1,3002-1,3096_aligned.jls")
ms_dict = [k => load(joinpath(ms_dir, v)) for (k,v) in ms_filenames];

Loading data from /home/ubuntu/davit/standard_224_annotated/1,3002-1,3096_aligned.jls
Loaded MeshSet from /home/ubuntu/davit/standard_224_annotated/1,3002-1,3096_aligned.jls
Loading data from /home/ubuntu/davit/standard_224_annotated/1,2-1,96_aligned.jls
Loaded MeshSet from /home/ubuntu/davit/standard_224_annotated/1,2-1,96_aligned.jls
Loading data from /home/ubuntu/davit/standard_224_annotated/1,1002-1,1096_aligned.jls
Loaded MeshSet from /home/ubuntu/davit/standard_224_annotated/1,1002-1,1096_aligned.jls
Loading data from /home/ubuntu/davit/standard_224_annotated/1,2002-1,2096_aligned.jls
Loaded MeshSet from /home/ubuntu/davit/standard_224_annotated/1,2002-1,2096_aligned.jls


### Compare meshsets  
A good kernel should create _more good matches_ and make it _easier to remove bad matches_.  

_More good matches_ means:  
1. a kernel finds good correspondences in locations where another kernel might not be able to find any.  
1. there is better coverage of matches (a more consistent density).   

_Easier to remove bad matches_ means:   
1. there are fewer bad matches overall.
1. we can filter out bad matches with an easily tuned filter (less need for manual intervention).  

To start evaluating these two criteria, we will inspect the f-score between pairs of kernels, as well as number of matches away from a perfect set of correspondences.

In [3]:
ms_dict["original"].matches[1].properties["params"]

Dict{ASCIIString,Any} with 7 entries:
  "mesh"     => Dict("mesh_length"=>400)
  "registry" => Dict("global_offsets"=>true)
  "solve"    => Dict{ASCIIString,Any}("ftol_cg"=>1.0e-8,"use_conjugate_gradient…
  "filter"   => Dict{Any,Any}()
  "render"   => Dict{Any,Any}()
  "match"    => Dict{ASCIIString,Any}("depth"=>1,"block_r"=>336,"bandpass_sigma…
  "review"   => Dict{ASCIIString,Any}("too_few_corresps"=>(:count_correspondenc…

In [4]:
p1 = get_properties(ms_dict["original"].matches[1], "src_range");
p2 = get_properties(ms_dict["net_adj"].matches[1], "src_range");

In [5]:
p12 = union(Set(p1), Set(p2))
p12 == Set(p1)

true

In [18]:
"""
Count TP, FP, FN between matchA & matchB (ground truth: matchA)
"""
function compare_matches(matchA, matchB)
    pA = Set(get_filtered_properties(matchA, "src_range"));
    pB = Set(get_filtered_properties(matchB, "src_range"));
    tp = length(intersect(pA, pB))
    fn = length(setdiff(pA, pB))
    fp = length(setdiff(pB, pA))
    return [tp, fn, fp]
end

"""
Compute F1-score
"""
function compute_f1(tp, fn, fp)
    return 2*tp / (2*tp + fn + fp)
end

"""
Compute F1-score from table of tp, fn, fp
"""
function compute_f1(pn)
    f1_array = zeros(size(pn,1))
    for i in 1:size(pn,1)
        f1_array[i] = compute_f1(pn[i,:]...)
    end
    f1 = compute_f1(sum(pn,1)[:]...)
    return f1, f1_array
end

compute_f1 (generic function with 2 methods)

In [8]:
comparisons = [("original", "net_adj"),
                ("original", "net_across"),
                ("original", "bandpass"),
                ("net_adj", "net_across"),
                ("net_adj", "bandpass"),
                ("net_across", "bandpass")];

In [9]:
"""
Count TP, FP, FN between all corresponding matches (ground truth: msA)
"""
function compare_meshsets(msA, msB)
    pn = zeros(Int64, length(msA.matches), 3)
    for (k, (matchA, matchB)) in enumerate(zip(msA.matches, msB.matches))
        pn[k,:] = [compare_matches(matchA, matchB)...]
    end
    return pn
end

compare_meshsets (generic function with 1 method)

In [19]:
pn_results = Dict()
for (nameA, nameB) in comparisons
    pn = compare_meshsets(ms_dict[nameA], ms_dict[nameB])
    pn_results[(nameA, nameB)] = pn
end

In [20]:
f1_results = Dict()
for (name_pair, pn) in pn_results
    f1_results[name_pair] = compute_f1(pn)
end

Matches where another kernel had matches where the net did not

In [34]:
pn = pn_results["net_adj","bandpass"]
collect(1:size(pn,1))[pn[:,3] .> 1]
pn[pn[:,3] .> 1,:]

2x3 Array{Int64,2}:
 1573  0  2
 1461  0  2

In [35]:
pn = pn_results["original", "net_adj"]
collect(1:size(pn,1))[pn[:,2] .> 1]
pn[pn[:,2] .> 1,:]

2x3 Array{Int64,2}:
 1506  2  4
 1566  2  7

In [38]:
function list_discrepancies(matchA, matchB)
    pA = Set(get_filtered_properties(matchA, "src_range"));
    pB = Set(get_filtered_properties(matchB, "src_range"));
    fn = setdiff(pA, pB)
    fp = setdiff(pB, pA)
    return fn, fp
end

list_discrepancies (generic function with 1 method)

In [72]:
_, d = list_discrepancies(ms_dict["net_adj"].matches[67], ms_dict["bandpass"].matches[67])
d

Set(Any[(14613:15001,1:437),(8724:9396,11965:12637)])

In [51]:
ms_names = ["original", "net_adj", "net_across", "bandpass"]
pn_table = zeros(Int64, length(ms_names), length(ms_names), 3)
f1_table = zeros(length(ms_names), length(ms_names))
for (i, nameA) in enumerate(ms_names)
    for (j, nameB) in enumerate(ms_names)
        if (nameA, nameB) in keys(f1_results)
            pn_table[i,j,:] = sum(pn_results[(nameA, nameB)],1)
            f1_table[i,j] = f1_results[(nameA, nameB)][1]
        end
    end
end

In [63]:
s = collect([1,2,4])
println(ms_names[s])
for (k, t) in enumerate(["tp", "fn", "fp"])
    println(t)
    println(pn_table[s,s,k])
end
println("f1")
println(round(f1_table[s,s], 4))

ASCIIString["original","net_adj","bandpass"]
tp
[0 143672 143657
 0 0 144342
 0 0 0]
fn
[0 12 27
 0 0 94
 0 0 0]
fp
[0 764 702
 0 0 17
 0 0 0]
f1
[0.0 0.9973 0.9975
 0.0 0.0 0.9996
 0.0 0.0 0.0]


In [73]:
function count_errors(ms, match)
    src_index = match.src_index
    mesh = get_mesh(ms, match.src_index)
    attempted_matches = length(mesh.src_nodes)
    possible_matches = length(get_properties(match, "src_range"))
    good_matches = length(get_filtered_properties(match, "src_range"))
    return [attempted_matches, possible_matches, good_matches]
end

count_errors (generic function with 1 method)

In [74]:
error_results = Dict()
for (name, ms) in ms_dict
    error_results[name] = zeros(Int64, 3)
    for match in ms.matches
        error_results[name] += count_errors(ms, match)
    end
end

In [75]:
error_results

Dict{Any,Any} with 4 entries:
  "bandpass"   => [170798,144500,144359]
  "original"   => [170798,144500,143684]
  "net_adj"    => [170798,144539,144436]
  "net_across" => [163530,138302,138105]

In [79]:
for (k,v) in error_results
    println("$k \t $(144500 - v[3])") # hardcode at 144500, because the net kernel created areas that it could match
end

bandpass 	 141
original 	 816
net_adj 	 64
net_across 	 6395


In [90]:
ms_dict["original"].matches[1].correspondence_properties[1]["xcorr"]["difference"]

Dict{Int64,Any} with 3 entries:
  10 => 0.15713293413196533
  5  => 0.1131847568745201
  15 => 0.17025758892888013